Exercise 1: Transportation Problem [15 marks]
A customer has contacted Saga Logistics for transporting electic vehicles (EVs) from warehouses
to markets. There are eight warehouses, each of which has some truck-loads of EVs in stock. These
EVs must be transported to eight markets for satisfying the demand. You are interning at Saga
Logistics and you are given the task of finding out how many EVs must be transported from each
warehouse to each market.

1. [R] Let C[w, m] be the cost of transporting one truck-load of EVs from w to m. Write
a general optimization problem to minimize the total transportation cost. Use appropriate
notations and define appropriate sets to be used in the optimization problem.

$\text{Solution : -} $ \\

Let's take N warehouses and M markets. \\
so, for N warehouses we have N avail. stock $s_{i} \ \text{where} i \in \{1,2,...,N\}$ \\
and,  for M markets we have M demand $d_{j} \ \text{where} j \in \{1,2,...,M\}$ \\

so, mathematical formulation for general optimization problem is \\

$ \text{Min} \ \ \sum_{i=1}^{N} \sum_{j=1}^{M}C[i,j]x_{ij}  \\
\ \ \ \ \  \ \  \ \sum_{i=1}^{N}x_{ij} = d_{j} \ \text{Where} \  j \in \{1,2,3,...,M\} \\
\ \ \ \ \ \ \ \ \ \sum_{j=1}^{M}x_{ij} \leq s_{i} \text{Where} \  i \in \{1,2,3,...,N\} \\ and \ x_{ij} \ \ \text{should be NonNegativeIntegers}.$

where, $C[i, j]$ be the cost of transporting one truck-load of EVs from $i$ wherehouse to $j$ market and corresponding variable is $x_{ij}$ denoting how much amount of EVs transporting from $i$ wherehouse  to $j$ market .



    

In [57]:
!pip install -q pyomo

In [58]:
from pyomo.environ import * 

In [59]:
import numpy as np

In [60]:
import pandas as pd

In [61]:
data_csvfile = pd.read_csv('lab6_ex1.csv')

In [62]:
data_csvfile

,Warehouses/Markets,Hassan,Jodhpur,Trichy,Simla,Vijayawada,Dibrugarh,Raipur,Agartala,Available Stock
0,Ahmedabad,827,817,1470,2272,1615,3143,1264,4465,350
1,Bengaluru,559,1832,751,3772,637,3773,1856,5459,185
2,Chennai\t,110,2443,577,3865,798,3521,1984,5303,305
3,Delhi,2943,1471,4676,1337,2702,2741,2482,4148,180
4,Kolkata,3742,3089,2931,3199,2226,1739,1691,2128,205
5,Lucknow,2353,1160,3295,368,2369,2365,1293,3642,175
6,Mumbai,1356,1617,2045,2941,2334,4151,1937,4287,325
7,Hyderabad,195,1862,1331,2921,402,2825,563,4441,370
8,Demand,205,300,425,270,375,225,185,110,2095


In [63]:
# create a model
model = ConcreteModel()

In [64]:
M = len(data_csvfile.index)-1
N = len(data_csvfile.columns)-2

In [65]:
row_indices = range(M)
col_indices = range(N)

In [66]:
model.x = Var(row_indices, col_indices, domain= NonNegativeReals)

In [67]:
model.objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [68]:
model.constraints = ConstraintList()

In [69]:
#supply constraint 
for i in range(M):
  model.constraints.add(expr=sum(model.x[i,j] for j in col_indices) <= data_csvfile.iloc[i,N+1]) 

#demand constraint
for j in range(N):
  model.constraints.add(expr=sum(model.x[i,j] for i in row_indices) == data_csvfile.iloc[M,j+1])  

In [70]:
!apt-get install -y -qq coinor-cbc

In [71]:
opt_cbc = SolverFactory('cbc')

In [72]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [73]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0: 
      print('Number of transporting electic vehicles (EVs) from warehouses',data_csvfile['Warehouses/Markets'][i],' to markets ',data_csvfile.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraints.display()


Objective =  2192870.0

Decision Variables
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Jodhpur : 300.0
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Dibrugarh : 45.0
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Raipur : 5.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Trichy : 120.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Vijayawada : 65.0
Number of transporting electic vehicles (EVs) from warehouses Chennai	  to markets  Trichy : 305.0
Number of transporting electic vehicles (EVs) from warehouses Delhi  to markets  Simla : 95.0
Number of transporting electic vehicles (EVs) from warehouses Delhi  to markets  Dibrugarh : 85.0
Number of transporting electic vehicles (EVs) from warehouses Kolkata  to markets  Dibrugarh : 95.0
Number of transporting electic vehicles (EVs) from warehouses

8. [R] Suppose the Lucknow-Simla link is disrupted and no transportation is possible on the
Lucknow-Simla links. Without changing the pyomo model, how will you solve this problem?
You are only allowed to change the .csv file.


$\text{Solution :-}$ Because of no transportation is possible on the Lucknow-simla Links so for that we can assign very high cost to this route so that under consideration of taking minimun cost route to minimize total cost the route with very high cost is not going to consider.

In [74]:
data_csvfile.loc[5,'Simla'] = 1000000000000

In [75]:
model.objective.deactivate()

In [76]:
model.new_objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [77]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


9. [R] Report the new solution value and the nonzero flows in the network. Comment on your
observations

$\text{Solution :-}$ Before increasing the cost of Lucknow-simla link , optimal solution showing that we need to send 175 Evs to minimize total cost but because after disrupted because no transportation happening between Lunknow-simla link so total cost is increased.

In [78]:
# display solution
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0:
      print('Number of transporting electic vehicles (EVs) from warehouses',data_csvfile['Warehouses/Markets'][i],' to markets ',data_csvfile.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraints.display()


Objective =  2380570.0

Decision Variables
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Jodhpur : 300.0
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Raipur : 50.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Trichy : 120.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Vijayawada : 65.0
Number of transporting electic vehicles (EVs) from warehouses Chennai	  to markets  Trichy : 305.0
Number of transporting electic vehicles (EVs) from warehouses Delhi  to markets  Simla : 180.0
Number of transporting electic vehicles (EVs) from warehouses Kolkata  to markets  Dibrugarh : 95.0
Number of transporting electic vehicles (EVs) from warehouses Kolkata  to markets  Agartala : 110.0
Number of transporting electic vehicles (EVs) from warehouses Lucknow  to markets  Dibrugarh : 130.0
Number of transporting electic vehicles (EVs) from warehou

10. [R] Suppose in addition to Lucknow-Simla link, the Kolkata-Dibrugarh link and Delhi-Raipur
link are disrupted and no transportation is possible along these links as well. Make appropriate
changes only to the .csv file and solve the model.

$\text{Solution :-}$ 

In [79]:
data_csvfile.loc[4,'Dibrugarh'] = 1000000000000
data_csvfile.loc[3,'Raipur'] = 1000000000000


In [80]:
model.new_objective.deactivate()

In [81]:
model.new2_objective = Objective(expr=sum(data_csvfile.iloc[i,j+1]*model.x[i,j] for i in row_indices for j in col_indices),sense=minimize)

In [82]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


11. [R] Report the new solution value and the nonzero flows in the network. Explain your
observations.


$\text{Solution :-}$ by seeing by solution we can observe that  delhi-Raipur link is already not using for transporting Evs but Kolkata-dibrugarh link 95 Evs are transported not that's by when we not considering these links for transportion cost is increased. we see even in above solution delhi-Raipur is not considering for transportion than also we need to restrict because if we not restrict may be this link is used for transportion.

In [83]:
# display solution
print('\nObjective = ', model.new2_objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value !=0:
      print('Number of transporting electic vehicles (EVs) from warehouses',data_csvfile['Warehouses/Markets'][i],' to markets ',data_csvfile.columns[j+1],':', model.x[i,j].value)

print('\nConstraints')
model.constraints.display()


Objective =  2518200.0

Decision Variables
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Jodhpur : 300.0
Number of transporting electic vehicles (EVs) from warehouses Ahmedabad  to markets  Dibrugarh : 50.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Trichy : 120.0
Number of transporting electic vehicles (EVs) from warehouses Bengaluru  to markets  Vijayawada : 65.0
Number of transporting electic vehicles (EVs) from warehouses Chennai	  to markets  Trichy : 305.0
Number of transporting electic vehicles (EVs) from warehouses Delhi  to markets  Simla : 180.0
Number of transporting electic vehicles (EVs) from warehouses Kolkata  to markets  Raipur : 95.0
Number of transporting electic vehicles (EVs) from warehouses Kolkata  to markets  Agartala : 110.0
Number of transporting electic vehicles (EVs) from warehouses Lucknow  to markets  Dibrugarh : 175.0
Number of transporting electic vehicles (EVs) from warehou